In [2]:
       
       # -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
import pandas as pd
import numpy as np
import math as mh 
from math import exp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
from scipy.stats import shapiro
import statistics
import pandas as pd
import glob
import csv 

In [10]:

path = r'C:\\Users\\BANG\\Desktop\\Dat\\' # use your path
all_files = glob.glob(path + "/excel_*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df.drop_duplicates(subset=['<Ticker>', '<DTYYYYMMDD>'], keep=False)



df.to_csv(r"C:\\Users\\BANG\\Desktop\\Dat\\data_frame.csv", index=True, header=True,encoding='utf-8-sig')


def Bayes(dt):    
   dt = dt.sort_values(by=['<DTYYYYMMDD>'], ascending=True)
   dt["return"]=np.log(dt["<CloseFixed>"]/dt["<CloseFixed>"].shift(periods=1))
   n = 100
   close_list = [ x for x in (dt["<CloseFixed>"].to_numpy()) ]
   return_list =[ x for x in (dt["return"].to_numpy()) ]
        
   cd_list = list()
   ct_list = list()
   close_predict_list = list()
        
   cd_list.append(1)
   ct_list.append(1)
   close_predict_list.append(1)
        
   mean_llh_list = list()
   var_llh_list = list()
   std_llh_list = list()
        
   mean_llh_list.append(1)
   var_llh_list.append(1)
   std_llh_list.append(1)
        
   mean_prior_list = list()
   var_prior_list = list()
        
   mean_prior_list.append(1)
   var_prior_list.append(1)
        
   mean_post_list = list()
   std_post_list = list()
        
   mean_post_list.append(1)
   std_post_list.append(1)
        
   for i  in range(1,len(dt)-n):
       ## Hàm hợp l
       dt_temp_llh=dt[1:i]
       mean_llh = dt_temp_llh["return"].mean()
       var_llh = dt_temp_llh["return"].var()/i
       std_llh = dt_temp_llh["return"].std()
            
       mean_llh_list.append(mean_llh)
       var_llh_list.append(var_llh)
       std_llh_list.append(std_llh)
            
       ## Hàm tiên nghiệm
       prior_list = return_list[1:n+i]
       prior_list.sort(reverse = True)
        
       prior_list = prior_list[int(len(prior_list) * 0.25): int(len(prior_list)*0.75)]
        
       mean_prior = statistics.mean(prior_list)
       var_prior = statistics.variance(prior_list)
            
       mean_prior_list.append(mean_prior)
       var_prior_list.append(var_prior)
            
       ##Hàm hậu nghiệm
       mean_post = (mean_prior * var_llh + i*mean_llh*var_prior) / (var_llh*i + i*var_prior)
       std_post = (var_prior*var_llh*i)/ (var_llh*i + i*var_prior)
            
       mean_post_list.append(mean_post)
       std_post_list.append(std_post)
            
       ##Dự báo
       close_predit = exp(mean_post) *  close_list[i]
       close_predict_list.append(close_predit)
            
       std_pre= statistics.stdev(close_predict_list)
        
       #cực đại
       cd = exp(mean_post + np.sqrt(np.square(std_post) + np.square(std_llh)) ) *  close_list[i]
       cd_list.append(cd)
        
       #cực tiểu
       ct = exp(mean_post - np.sqrt(np.square(std_post) + np.square(std_llh)) ) *  close_list[i]
       ct_list.append(ct)
        
   dt_temp =dt
   dt_temp=dt_temp[1:len(dt)-n+1]
   
   len(close_predict_list)
   
   dt_temp["mean_llh"] = mean_llh_list
   dt_temp["var_llh"] = var_llh_list
   dt_temp["std_llh"] = std_llh_list
        
   dt_temp["mean_prior"] = mean_prior_list
   dt_temp["var_prior"] = var_prior_list
   
   dt_temp["mean_post"] = mean_post_list
   dt_temp["std_post"] = std_post_list
   
   
   
   dt_temp["close_list"] = close_list[:len(close_list)-100]  
   dt_temp["close_predict"] = close_predict_list
   
   #dt_temp["ctsf"] =float(exp(dt_temp["mean_post"]*(1+a) + dt_temp["std_post"])) 

   #dt_temp["ct"] =float(exp(dt_temp["mean_post"]*(1+a) + dt_temp["std_post"])) * dt_temp["<CloseFixed>"] 
   dt_temp["cd"] = cd_list
        

        
   dt_temp["mean_post"] = mean_post_list
   dt_temp["std_post"] = std_post_list
   
   dt_temp["ct"] = ct_list
   
        
   ##Check
   dt_temp["check"] =  (dt_temp["ct"] <= dt_temp["<CloseFixed>"]) *  (dt_temp["cd"] >= dt_temp["<CloseFixed>"]) 
          
   ## XUẤT KẾT QUẢ RA CSV
   #dt_temp.to_csv(r"D:\\99_LUAN_VAN\\01 CODE\\DATA_STOCK\temp_kq20200110.csv", index=True, header=True)
   return dt_temp


arrTicket=df['<Ticker>'].unique()







,<Ticker>,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,Index,T+1,T+2,T+3
0,ABI,20200317,26.70,26.80,24.60,26.80,1000,26.7,26.8,24.6,26.8,0,700,200,1,,,
1,ABI,20200316,27.90,27.90,24.30,24.30,2000,27.9,27.9,24.3,24.3,0,200,100,2,,,
2,ABI,20200313,24.00,25.00,24.00,25.00,1200,24.0,25.0,24.0,25.0,0,300,0,3,,,
3,ABI,20200312,27.40,27.40,25.00,25.00,1400,27.4,27.4,25.0,25.0,0,100,0,4,,,
4,ABI,20200311,26.60,27.90,26.00,26.00,1300,26.6,27.9,26.0,26.0,0,200,0,5,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2060,ABI,20090727,10.09,10.09,8.31,8.30,5050,27.1,27.1,22.3,22.3,0,0,0,2061,,,
2061,ABI,20090724,9.19,9.19,9.19,9.20,110,24.7,24.7,24.7,24.7,0,0,0,2062,,,
2062,ABI,20090723,8.82,8.82,7.44,8.82,60,23.7,23.7,20.0,23.7,0,0,0,2063,,,
2063,ABI,20090721,8.04,8.04,8.04,8.04,2000,21.6,21.6,21.6,21.6,0,0,0,2064,,,


In [24]:
#B1
df['T+1']=''
df['T+2']=''
df['T+3']=''

df['T+1']=df['Index']%1
df['T+2']=df['Index']%2
df['T+3']=df['Index']%3

df1=df[df['T+1']==1]
df2=df[df['T+2']==1]
df3=df[df['T+3']==1]


df2

,<Ticker>,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,Index,T+1,T+2,T+3
0,ABI,20200317,26.70,26.80,24.60,26.80,1000,26.7,26.8,24.6,26.8,0,700,200,1,0,1,1
2,ABI,20200313,24.00,25.00,24.00,25.00,1200,24.0,25.0,24.0,25.0,0,300,0,3,0,1,0
4,ABI,20200311,26.60,27.90,26.00,26.00,1300,26.6,27.9,26.0,26.0,0,200,0,5,0,1,2
6,ABI,20200309,26.00,26.00,24.90,24.90,14800,26.0,26.0,24.9,24.9,0,8200,7800,7,0,1,1
8,ABI,20200305,26.70,26.70,26.30,26.30,10100,26.7,26.7,26.3,26.3,0,3600,9000,9,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056,ABI,20090731,12.44,12.44,10.20,10.20,220,33.4,33.4,27.4,27.4,0,0,0,2057,0,1,2
2058,ABI,20090729,11.29,11.29,9.31,11.28,820,30.3,30.3,25.0,30.3,0,0,0,2059,0,1,1
2060,ABI,20090727,10.09,10.09,8.31,8.30,5050,27.1,27.1,22.3,22.3,0,0,0,2061,0,1,0
2062,ABI,20090723,8.82,8.82,7.44,8.82,60,23.7,23.7,20.0,23.7,0,0,0,2063,0,1,2


In [23]:
df3

,<Ticker>,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,Index,T+1,T+2,T+3
0,ABI,20200317,26.70,26.80,24.60,26.80,1000,26.7,26.8,24.6,26.8,0,700,200,1,0,1,1
3,ABI,20200312,27.40,27.40,25.00,25.00,1400,27.4,27.4,25.0,25.0,0,100,0,4,0,0,1
6,ABI,20200309,26.00,26.00,24.90,24.90,14800,26.0,26.0,24.9,24.9,0,8200,7800,7,0,1,1
9,ABI,20200304,27.50,27.50,26.50,26.50,3500,27.5,27.5,26.5,26.5,0,300,0,10,0,0,1
12,ABI,20200228,27.50,29.50,27.00,27.90,5000,27.5,29.5,27.0,27.9,0,300,9600,13,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,ABI,20090807,14.56,14.56,14.56,14.56,10,39.1,39.1,39.1,39.1,0,0,0,2053,0,1,1
2055,ABI,20090803,12.06,12.06,9.90,12.06,40,32.4,32.4,26.6,32.4,0,0,0,2056,0,0,1
2058,ABI,20090729,11.29,11.29,9.31,11.28,820,30.3,30.3,25.0,30.3,0,0,0,2059,0,1,1
2061,ABI,20090724,9.19,9.19,9.19,9.20,110,24.7,24.7,24.7,24.7,0,0,0,2062,0,0,1


In [11]:
df0 = df.groupby('<Ticker>')
df1 = df0.get_group('ABI').set_index('<DTYYYYMMDD>').sort_values(by=['<DTYYYYMMDD>'],ascending=True)


df_result = (Bayes(df1)).iloc[0:0]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pand

In [12]:
df_result

,<Ticker>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,...,std_llh,mean_prior,var_prior,mean_post,std_post,close_list,close_predict,cd,ct,check
<DTYYYYMMDD>,,,,,,,,,,,,,,,,,,,,,


In [ ]:
for index in range(len(arrTicket)):
    #print(arrTicket[index])
    df_tempt = df0.get_group(arrTicket[index]).sort_values(by=['<DTYYYYMMDD>'],ascending=False).set_index('<DTYYYYMMDD>')
    #df1_result = df_null
    df1_result = Bayes(df_tempt)
    #print(arrTicket[index])
     #print(arrTicket[index])
    #df1_result
    df_result = df_result.append(df1_result)
    #df1_result.to_csv(f"D:\\99_LUAN_VAN\\01 CODE\\DATA_STOCK\\ExportSourceDataFame\\df_result_{arrTicket[index]}.csv", index=True, header=True,encoding='utf-8-sig')
    df_result.to_csv(f"C:\\Users\\BANG\\Desktop\\Dat\\ExportSourceDataFame\\df_result.csv", index=True, header=True,encoding='utf-8-sig')
            
 